In [ ]:
# loop through the nwm land surface files (LDAS)
# and combine all the years for each basin.

In [1]:
import copy
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time
import tools
from tqdm import tqdm

In [9]:
data_v2_dir = '/.../v2/'
data_v2_dir_cam = '/.../v2/LDAS/camels_basins/'
proj_data_dir = '/.../data/'
combine_years = True
dis_cumulate = True
average_days = Ture
start_end_year_list = [1993, 2018]
rest_of_years_list = [y for y in range(start_end_year_list[0]+1, start_end_year_list[1])]

In [3]:
# Here I am going to load all of the individual years with the collated data, 
# then put it together.
df = {}
# Load these as a samples no matter what, we want their start/end dates:
file_name_prefix = 'dynamic_features_nwm_ldas_'
for y in start_end_year_list:
    with open(data_v2_dir_cam+file_name_prefix+str(y)+'.p', 'rb') as pf:
        df[y] = pickle.load(pf)

# Then the rest of the individual years, if needed
if combine_years:
    for y in rest_of_years_list:
        with open(data_v2_dir_cam+file_name_prefix+str(y)+'.p', 'rb') as pf:
            df[y] = pickle.load(pf)
else:
    print('LDAS years not loaded here')

LDAS years not loaded here


In [4]:
basins = list(df[1993].keys())
sample_basin = basins[0]
firstdt = df[start_end_year_list[0]][sample_basin].index.values[0]
lastd = df[start_end_year_list[1]][sample_basin].index.values[-1]
date_listn= pd.date_range(start=firstdt, end=lastd, freq='3H')
feature_names = list(df[1993][sample_basin].columns.values)
n_dates = date_listn.shape[0]
n_features = len(feature_names)

In [5]:
# Set up the dictionary with a dataframe for each basin,
# then append in the year values from our files.
if combine_years:
    ldas = {b:pd.DataFrame(columns=feature_names) for b in basins}
    for b in basins:
        for y in df.keys():
            ldas[b] = ldas[b].append(df[y][b])
        ldas[b] = ldas[b].sort_index()
else:
    print('LDAS years not combined here')

LDAS years not combined here


In [6]:
# Remove duplicate values, 
# since time zero shows up at the beginning and end of year, 
# leading to overlap.
if combine_years:
    print(ldas[sample_basin].iloc[2914:2928])
    for b in basins:
        ldas[b] = ldas[b].sort_index()
        # dropping duplicate values 
        ldas[b].drop_duplicates(keep='first',inplace=True) 
else:
    print('Duplicate date/times not removed here')

Duplicate date/times not removed here


In [7]:
# Replace the -999.. values with non numbers
if combine_years:
    for b in tqdm(basins):
        ldas[b] = ldas[b].apply(lambda x: [y if y > -990 else np.nan for y in x])
else:
    print('Bad values not filled with NaNs here')

Bad values not filled with NaNs here


In [11]:
# Mark the dates/times for a reset.
if dis_cumulate:
    good_reset_dates = {}
    previous_reset_date_available = False
    for b_count, b in enumerate(basins):
        reset_dates = []
        for i, d in enumerate(ldas[b].index.values):
            if i > 1:
                d_1_back = ldas[b].index.values[i-1]
                ts_day = pd.to_datetime(d).day
                ts_hour = pd.to_datetime(d).hour
                ts_month = pd.to_datetime(d).month

                if np.isnan(ldas[b].loc[d_1_back, 'ACCET']) or np.isnan(ldas[b].loc[d, 'ACCET']):
                    continue
                if np.isnan(ldas[b].loc[d_1_back, 'UGDRNOFF']) or np.isnan(ldas[b].loc[d, 'UGDRNOFF']):
                    continue

                elif ldas[b].loc[d, 'UGDRNOFF'] - ldas[b].loc[d_1_back, 'UGDRNOFF'] == 0:
                    break
                elif ldas[b].loc[d, 'UGDRNOFF'] - ldas[b].loc[d_1_back, 'UGDRNOFF'] < 0:
                    reset_dates.append(d)

        if i > ldas[b].index.values.shape[0] - 2:
            good_reset_dates[b] = reset_dates
            if previous_reset_date_available:
                if reset_dates != previous_reset_date:
                    print('reset dates differ at basins {}'.format(b))
            previous_reset_date = reset_dates
            previous_reset_date_available = True

        if b_count > 10:
            break
    print(good_reset_dates)
else:
    print('Reset dates not set here')

Reset dates not set here


In [12]:
# Copy ldas, because we want to save the cumulative values, just in case.
if dis_cumulate:
    ldas2 = copy.deepcopy(ldas)
    # subtract out the time step values from the features with cumulative sums (e.g., ACCET & UGDRNOFF)
    for b_count, b in enumerate(basins):
        for i, d in enumerate(ldas[b].index.values):
            d_1_back = ldas[b].index.values[i-1]
            if i > 1:
                for dfn in ['ACCET', 'UGDRNOFF']: # dfn = dynamic feature name
                    if np.isnan(ldas[b].loc[d_1_back, dfn]):
                        ldas2[b].loc[d, dfn] = np.nan
                    elif d in reset_dates:
                        ldas2[b].loc[d, dfn] = ldas[b].loc[d, dfn]
                    elif d not in reset_dates:
                        ldas2[b].loc[d, dfn] = ldas[b].loc[d, dfn] - ldas[b].loc[d_1_back, dfn]

        # Save file with the 3 hour intervals
        with open(data_v2_dir+'ldas_v2_3h.p', 'wb') as pf:
            pickle.dump(ldas2, pf, protocol=pickle.HIGHEST_PROTOCOL)
else:
    # Load file with the 3 hour intervals
    print('loading in LDAS data as ldas2')
    with open(data_v2_dir+'ldas_v2_3h.p', 'rb') as pf:
        ldas2 = pickle.load(pf)

loading in LDAS data as ldas2


In [13]:
# Now average the basins by day, instead of 3 hour intervals.
if average_days:
    def daily_average(df):
        return(df.groupby(pd.Grouper(freq='1D')).mean())
    ldas_v2_1day_ave = {k: daily_average(v) for (k, v) in ldas2.items()}
    
    for b in basins:
        ldas_v2_1day_ave[b].index.name = 'time'
    
    with open(data_v2_dir+'nwm_ldas_v2_1d.p', 'wb') as f:
        pickle.dump(ldas_v2_1day_ave, f)
else:
    print('Load data averaged by day')
    with open(data_v2_dir+'nwm_ldas_v2_1d.p', 'rb') as pf:
        ldas_v2_1day_ave = pickle.load(pf)

Load data averaged by day
